In [9]:

import pandas as pd
import talib
import numpy as np
import matplotlib.pyplot as plt
import os
import logging
#import time
#import uuid
#from datetime import datetime, timedelta
#Performence:
from config_parameters import MARKET_DATA_DICT, DATA_TABLE_FILE, STRATEGIES_TABLE_FILE, SIMULATION_DATA_DICT, LOGS_FOLDER
from config_simulation import COMMISSION_RATE , RISK_SIZE, PRINT_DAILY_UPDATE, PRINT_LOGS, START_BALANCE



interval_mapping = {
    '1M':  {'name': '1M',  'dataOffset': pd.Timedelta(days=31)},
    '2w':  {'name': '1w',  'dataOffset': pd.Timedelta(days=14)},
    '1w':  {'name': '1w',  'dataOffset': pd.Timedelta(days=7)},
    '3d':  {'name': '3d',  'dataOffset': pd.Timedelta(days=3)},
    '2d':  {'name': '2d',  'dataOffset': pd.Timedelta(days=2)},
    '1d':  {'name': '1d',  'dataOffset': pd.Timedelta(days=1)},
    '12h': {'name': '12h', 'dataOffset': pd.Timedelta(hours=12)},
    '8h':  {'name': '8h',  'dataOffset': pd.Timedelta(hours=8)},
    '7h':  {'name': '7h',  'dataOffset': pd.Timedelta(hours=7)},
    '6h':  {'name': '6h',  'dataOffset': pd.Timedelta(hours=6)},
    '4h':  {'name': '4h',  'dataOffset': pd.Timedelta(hours=4)},
    '2h':  {'name': '2h',  'dataOffset': pd.Timedelta(hours=2)},
    '1h':  {'name': '1h',  'dataOffset': pd.Timedelta(hours=1)},
    '30m': {'name': '30m', 'dataOffset': pd.Timedelta(minutes=30)},
    '15m': {'name': '15m', 'dataOffset': pd.Timedelta(minutes=15)},
    '10m': {'name': '10m', 'dataOffset': pd.Timedelta(minutes=10)},
    '5m':  {'name': '5m',  'dataOffset': pd.Timedelta(minutes=5)},
    '3m':  {'name': '3m',  'dataOffset': pd.Timedelta(minutes=3)},
    '1m':  {'name': '1m',  'dataOffset': pd.Timedelta(minutes=1)}
}




def add_williams_r(data):
    data['High_14'] = data['high'].rolling(window=14).max()
    data['Low_14'] = data['low'].rolling(window=14).min()
    data['Williams_%R'] = (data['High_14'] - data['close']) / (data['High_14'] - data['Low_14']) * -100
    return data


def calculate_stoch_rsi(data, period=14):
    min_rsi = data['RSI'].rolling(window=period).min()
    max_rsi = data['RSI'].rolling(window=period).max()
    data['Stoch_RSI'] = (data['RSI'] - min_rsi) / (max_rsi - min_rsi)
    return data


def calculate_stoch_rsi_k_d(data, rsi_period=14, k_period=3):
    data = calculate_stoch_rsi(data, period=rsi_period)
    data['K'] = data['Stoch_RSI'].rolling(window=k_period).mean() * 100
    data['D'] = data['K'].rolling(window=k_period).mean()
    return data


def add_price_action_indicators(data):
    data['Avg_Volume'] = data['volume'].rolling(window=40).mean()
    data['High_Low_Range'] = data['high'] - data['low']
    data['High_Volume'] = (data['volume'] > 1.8 * data['Avg_Volume']).astype(int)
    data['Long_Candle'] = (data['High_Low_Range'] > 1.2 * data['ATR']).astype(int)
    data['Bullish_Body'] = ((data['close'] - data['low']) > data['High_Low_Range'] * 0.7).astype(int)
    data['Bearish_Body'] = ((data['high'] - data['close']) > data['High_Low_Range'] * 0.7).astype(int)
    data['Bullish_Pattern'] = (data['High_Volume'] & data['Long_Candle'] & data['Bullish_Body']).astype(int)
    data['Bearish_Pattern'] = (data['High_Volume'] & data['Long_Candle'] & data['Bearish_Body']).astype(int)
    return data



def add_indicator_ema(data: pd.DataFrame, periods: list[int]) -> pd.DataFrame:
    """
    Adds EMA indicators for the specified periods to the DataFrame.

    :param data: DataFrame with 'close' column.
    :param periods: List of integers representing EMA periods.
    :return: DataFrame with added EMA columns.
    """
    for period in periods:
        column_name = f'EMA_{period}'
        data[column_name] = data['close'].ewm(span=period, adjust=False).mean()
    return data


def add_indicator_sma(data: pd.DataFrame, periods: list[int]) -> pd.DataFrame:
    """
    Adds SMA indicators for the specified periods to the DataFrame.

    :param data: DataFrame with 'close' column.
    :param periods: List of integers representing SMA periods.
    :return: DataFrame with added SMA columns.
    """
    for period in periods:
        column_name = f'SMA_{period}'
        data[column_name] = data['close'].rolling(window=period).mean()
    return data

def calculate_bluewaves_vwap(source, channel_length, average_length):
    esa = talib.EMA(source, channel_length)
    deviation = talib.EMA(np.abs(source - esa), channel_length)
    ci = (source - esa) / (0.015 * deviation)
    bw1 = talib.EMA(ci, average_length)
    bw2 = talib.SMA(bw1, 3)
    vwap = bw1 - bw2
    return bw1, bw2, vwap

def compute_cipher_source(df):
    hlc3 = (df['high'] + df['low'] + df['close']) / 3
    return calculate_bluewaves_vwap(hlc3, 9, 12)

def determine_trend(row):
    if row['m1'] > row['m2'] and row['m1_prev'] < row['m2_prev']:
        return 1
    elif row['m1'] < row['m2'] and row['m1_prev'] > row['m2_prev']:
        return -1
    else:
        return 0

def add_macd_indicators(df):
    short_window = 12
    long_window = 26
    signal_window = 9
    if 'MACD_Histogram' in df.columns and (df['MACD_Histogram'].last_valid_index() + 1 == len(df)):
        return df
    else:
        df['EMA12'] = df['close'].ewm(span=short_window).mean()
        df['EMA26'] = df['close'].ewm(span=long_window).mean()
        df['MACD'] = df['EMA12'] - df['EMA26']
        df['Signal_Line'] = df['MACD'].ewm(span=signal_window).mean()
        df['MACD_Histogram'] = df['MACD'] - df['Signal_Line']
        df['Signal'] = 0

        df.loc[(df['MACD'] > df['Signal_Line']) & (df['MACD'].shift(1) <= df['Signal_Line'].shift(1)), 'Signal_MACD'] = 1
        df.loc[(df['MACD'] < df['Signal_Line']) & (df['MACD'].shift(1) >= df['Signal_Line'].shift(1)), 'Signal_MACD'] = -1

        df['trend_MACD'] = df['MACD_Histogram'].apply(lambda x: 1 if x > 0 else -1)
        return df

def add_smma_indicator(df, length=7):
    if 'SMMA_7' in df.columns and (df['SMMA_7'].last_valid_index() + 1 == len(df)):
        return df
    else:
        df['SMMA_7'] = df['close'].ewm(alpha=1/length, adjust=False).mean()
        return df

def add_cipher_indicators(df, since=pd.to_datetime('2000-01-01 00:00:00')):
    required_columns = ['m1', 'm2', 'vW']
    if all(col in df.columns for col in required_columns):
        df = df.drop(['m1', 'm2', 'vW', 'm1_prev', 'm2_prev'], axis=1)

    bw1, bw2, vwap = compute_cipher_source(df)
    bw1.name = 'm1'
    bw2.name = 'm2'
    vwap.name = 'vW'

    df = df.join(bw1).join(bw2).join(vwap)
    df['m1_prev'] = df['m1'].shift(1)
    df['m2_prev'] = df['m2'].shift(1)
    df['MC_signal'] = 0

    trend_value = 0
    for i, row in df.iterrows():
        if row['m1'] > row['m2'] and row['m1_prev'] < row['m2_prev']:
            trend_value = 1
            df.at[i, 'MC_signal'] = 1
        elif row['m1'] < row['m2'] and row['m1_prev'] > row['m2_prev']:
            trend_value = -1
            df.at[i, 'MC_signal'] = -1
        df.at[i, 'trend_MC'] = trend_value

    return df


def calculate_true_range(df):
    """Calculate the True Range (TR)"""
    df['previous_close'] = df['close'].shift(1)
    df['TR'] = np.maximum(df['high'] - df['low'], 
                          np.maximum(abs(df['high'] - df['previous_close']), 
                                     abs(df['low'] - df['previous_close'])))
    return df

def sma(series, length):
    """Simple Moving Average (SMA)"""
    return series.rolling(window=length).mean()

def ema(series, length):
    """Exponential Moving Average (EMA)"""
    return series.ewm(span=length, adjust=False).mean()

def wma(series, length):
    """Weighted Moving Average (WMA)"""
    weights = np.arange(1, length + 1)
    return series.rolling(length).apply(lambda prices: np.dot(prices, weights) / weights.sum(), raw=True)

def rma(series, length):
    """Relative Moving Average (RMA)"""
    return series.ewm(alpha=1/length, adjust=False).mean()

def ma(source, length, ma_type):
    """Function for selecting the moving average type for RSI"""
    if ma_type == "SMA":
        return sma(source, length)
    elif ma_type == "EMA":
        return ema(source, length)
    elif ma_type == "WMA":
        return wma(source, length)
    elif ma_type == "SMMA (RMA)":
        return rma(source, length)
    else:
        raise ValueError("Unsupported MA type")

def ma_function(series, length, smoothing):
    if smoothing == "SMA":
        return sma(series, length)
    elif smoothing == "EMA":
        return ema(series, length)
    elif smoothing == "WMA":
        return wma(series, length)
    else:  # Default to RMA
        return rma(series, length)

def calculate_rsi(df, rsi_length=14, ma_type="SMA", ma_length=14, bb_mult=2):
    delta = df['close'].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)

    # Relative Strength (RS)
    avg_gain = rma(gain, rsi_length)
    avg_loss = rma(loss, rsi_length)
    rs = avg_gain / avg_loss

    # RSI calculation
    rsi = 100 - (100 / (1 + rs))

    # Moving average of RSI (based on selected MA type)
    rsi_ma = ma(rsi, ma_length, ma_type)



    df['RSI'] = rsi

    return df


def calculate_rsi_n(df, rsi_length=14):
    """
    Calculates RSI with a custom period and adds it as a new column 'RSI_{n}'.

    Parameters:
        df : pandas DataFrame with at least a 'close' column.
        rsi_length : int, period length for RSI calculation.

    Returns:
        DataFrame with a new column 'RSI_{n}'.
    """
    delta = df['close'].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)

    avg_gain = rma(gain, rsi_length)
    avg_loss = rma(loss, rsi_length)
    rs = avg_gain / avg_loss

    rsi = 100 - (100 / (1 + rs))
    df[f'RSI_{rsi_length}'] = rsi

    return df


def calculate_atr(df, length=14, smoothing="RMA"):
    """Average True Range (ATR) calculation"""
    df = calculate_true_range(df)
    df['ATR'] = ma_function(df['TR'], length, smoothing)
    return df

def calculate_obv(df, span1=1, span2=3):
    """
    Calculate the On-Balance Volume (OBV) indicator.

    :param df: DataFrame with 'close' and 'volume' columns
    :return: DataFrame with additional OBV-related columns
    """
    df['OBV'] = 0.0
    for i in range(1, len(df)):
        if df['close'][i] > df['close'][i - 1]:
            df.loc[i, 'OBV'] = df.loc[i - 1, 'OBV'] + df.loc[i, 'volume']
        elif df['close'][i] < df['close'][i - 1]:
            df.loc[i, 'OBV'] = df.loc[i - 1, 'OBV'] - df.loc[i, 'volume']
        else:
            df.loc[i, 'OBV'] = df.loc[i - 1, 'OBV']

    df['OBV_ExK1'] = df['OBV'].ewm(span=span1, adjust=False).mean()
    df['OBV_ExK2'] = df['OBV'].ewm(span=span2, adjust=False).mean()
    df['OBV_ExDiff'] = df['OBV_ExK1'] - df['OBV_ExK2']
    return df

def calculate_long_term_obv(df, period=10):
    """
    Calculates long-term OBV trend using exponential moving average.

    :param df: DataFrame with OBV-related columns
    :param period: Number of periods to include in the long-term trend
    :return: DataFrame with 'OBV_LT' column added
    """
    df['OBV_LT'] = df['OBV_ExDiff'].ewm(span=period, adjust=False).mean()
    return df



In [10]:
def get_local_data(ticker, interval, folder):
    """
    Loads local CSV data for a given ticker and interval.
    Converts 'index' column to 'timestamp' if necessary.
    """
    symbol = ticker.replace('/', '.')
    symbol_dir = os.path.join(folder, symbol)

    if not os.path.exists(symbol_dir):
        os.makedirs(symbol_dir)

    file_path = os.path.join(symbol_dir, f"{interval}.csv")

    try:
        df = pd.read_csv(file_path)
    except FileNotFoundError:
        print(f"❌ File not found: {file_path}")
        return None

    if 'timestamp' not in df.columns:
        if 'index' in df.columns:
            print(f"🔁 Replacing 'index' column with 'timestamp' in: {file_path}")
            df.rename(columns={'index': 'timestamp'}, inplace=True)
            df.to_csv(file_path, index=False)
        else:
            print(f"❌ Neither 'timestamp' nor 'index' found in: {file_path}")
            return None

    df = pd.read_csv(file_path, parse_dates=['timestamp'])
    return df


def process_data(df):
    """
    Applies a set of technical indicators and transformations to the input DataFrame.
    """
    data=df.copy()
    data = add_macd_indicators(data)
    data = calculate_obv(data)
    data = calculate_long_term_obv(data)
    data = calculate_atr(data, length=14, smoothing='RMA')
    data = add_macd_indicators(data)
    data = add_price_action_indicators(data)
    data = calculate_rsi(data)
    data = calculate_rsi_n(data,9)
    data = calculate_rsi_n(data,6)
    data = calculate_rsi_n(data,20)
    data = calculate_stoch_rsi_k_d(data)
    data = add_williams_r(data)
    data['Williams_%R_lag'] = data['Williams_%R'].shift(1)

    ema_periods = [5,10, 15,20, 50, 100, 150,  200, 500, 595]
    data = add_indicator_ema(data, ema_periods)

    sma_periods = [5, 10, 15, 20, 50, 100, 150, 200, 500, 592]
    data = add_indicator_sma(data, sma_periods)

    return data

In [11]:
class Trade:
    def __init__(self, trade_type, create_time, entry_time, entry_price, deposit, max_leverage, strategy, row, stop_loss=None):
        self.type = trade_type
        self.create_time = create_time
        self.entry_time = entry_time
        self.entry_price = entry_price
        self.deposit = deposit
        self.stop_loss = stop_loss
        self.strategy = strategy
        self.row = row

        # Trade sizing and leverage calculation
        if stop_loss:
            self.position_size = deposit / abs(entry_price - stop_loss)
            self.position_value = self.position_size * entry_price
            self.leverage = self.position_value / deposit

            if self.leverage > max_leverage:
                self.leverage = max_leverage
                self.position_value = deposit * max_leverage
                self.position_size = self.position_value / entry_price
        else:
            self.leverage = 1
            self.position_size = deposit / entry_price
            self.position_value = self.position_size * entry_price

        # Exit-related attributes
        self.exit_time = None
        self.exit_price = None
        self.exit_value = None
        self.pnl = None
        self.result = None
        self.status = "Active"  # Default status is 'Active'

    def set_exit(self, exit_time, exit_price):
        self.exit_time = exit_time
        self.exit_price = exit_price
        self.pnl = self.calculate_pnl()
        self.exit_value = self.calculate_exit_value()
        self.status = "Closed"

    def calculate_pnl(self):
        if self.type == 'L':  # Long
            return (self.exit_price - self.entry_price) * self.position_size
        elif self.type == 'S':  # Short
            return (self.entry_price - self.exit_price) * self.position_size

    def calculate_exit_value(self):
        commission = self.entry_price * self.position_size * COMMISSION_RATE 
        return self.deposit + self.pnl - 2 * commission

    def calculate_current_value(self, current_price):
        if self.type == 'L':
            return (current_price - self.entry_price) * self.position_size + self.deposit
        elif self.type == 'S':
            return (self.entry_price - current_price) * self.position_size + self.deposit

    def set_status(self, status):
        if status in ["Active", "Partially_Closed", "Closed"]:
            self.status = status
        else:
            raise ValueError("Invalid status. Must be one of: 'Active', 'Partially_Closed', 'Closed'.")

    def to_dict(self):
        base_data = {
            'Type': self.type,
            'Create Time': self.create_time,
            'Entry Time': self.entry_time,
            'Entry Price': self.entry_price,
            'Exit Time': self.exit_time,
            'Exit Price': self.exit_price,
            'Result': self.result,
            'P&L': self.pnl,
            'Leverage': self.leverage,
            'Deposit': self.deposit
        }

        # Include snapshot of market data at entry
        entry_market_snapshot = {
            f"{key} at entry": value for key, value in self.row.items()
        }

        return {**base_data, **entry_market_snapshot}

    def __str__(self):
        return f"{self.type} trade opened at {self.entry_time} with entry price {self.entry_price} and {self.position_size} units. Status: {self.status}"

In [12]:
class Order:
    def __init__(self, order_type, trade_type, create_time, deposit, max_leverage, strategy, row):
        self.order_type = order_type
        self.trade_type = trade_type              # Type of trade (e.g., "Long", "Short")
        self.create_time = create_time
        self.deposit = deposit                    # Capital allocated for the trade
        self.status = 'CREATED'                   # Initial status
        self.strategy = strategy                  # Strategy identifier
        self.max_leverage = max_leverage
        self.row = row                            # Market snapshot at creation time

    def fill(self, entry_time, entry_price):
        """
        Convert this order into a live trade.
        """
        new_trade = Trade(
            trade_type=self.trade_type,
            create_time=self.create_time,
            entry_time=entry_time,
            entry_price=entry_price,
            deposit=self.deposit,
            max_leverage=self.max_leverage,
            strategy=self.strategy,
            row=self.row
        )

        self.status = 'FILLED'
        return new_trade

    def __str__(self):
        return (
            f"Order: type={self.order_type}, trade_type={self.trade_type}, "
            f"created_at={self.create_time}, deposit={self.deposit}, strategy={self.strategy}"
        )

In [13]:
class Interval:
    def __init__(self, ticker, interval, data):
        self.ticker=ticker
        self.interval=interval
        self.data=data
        self.curr_idx=None 
        self.curr_date=None
        self.row=None
    def set_date(self, date):
        self.curr_idx = (self.data['timestamp'] - date).abs().idxmin()
        if self.curr_idx!=None:
            self.row=self.data.iloc[self.curr_idx]
            self.curr_date=self.row['timestamp']
           # logging.info(f"Date {date} found in data idx: {self.curr_idx}")
        #else:
            #logging.info(f"Date {date} not found in data")
    

    def set_next_date(self): # Next row if exists
        self.curr_idx+=1

        if 0 <= self.curr_idx < len(self.data):
            self.row = self.data.iloc[self.curr_idx]
            self.curr_date=self.row['timestamp']
        else:
            self.row = None  # If the index is out of range, set the row to None
            self.curr_date=None

In [14]:
def update_account_history(account_history: pd.DataFrame, timestamp, account_value):
    new_row = pd.DataFrame({'timestamp': [timestamp], 'account_value': [account_value]})
    
    if account_history.empty:
        return new_row

    return pd.concat([account_history, new_row], ignore_index=True)




def create_market_open_order(trade_type, deposit, row, strategy, max_levar=1):
    logging.info(f" row timestamp: {row['timestamp']} ")
    order_params = {
                            'order_type': 'OPEN MARKET',
                            'trade_type': trade_type,
                            'create_time': row['timestamp'],
                            'deposit': deposit,
                            'max_leverage': max_levar,
                            'strategy': strategy,
                            'row': row
                        }
    order = Order(**order_params)
    return order




def calculate_deposit(initial_balance, available_balance, stop_loss = None):
    if stop_loss:
        return initial_balance*RISK_SIZE
    else:
        return available_balance

In [15]:


def backtest(interval_dict, ticker, strategy, intervals,  start_date, end_date , start_balance=1000, position_risk=0.002, stop_loss=False):

    initial_balance=start_balance
    balance = start_balance  # Stan początkowy konta handlowego
    available_balance=balance
    account_worth= balance
    curr_date=start_date

    account_history = pd.DataFrame(columns=['timestamp', 'account_value'])
    Active_orders = []
    Closed_trades = []
    Active_trades = []
    active_long=0
    active_short=0

    logging.info(f"Starting singleinterval simulation {intervals} for ticker {ticker} strategy {strategy} for date: {start_date} - {end_date} ")
    #print(f"Starting singleinterval simulation {intervals} for ticker {ticker} strategy {strategy} for date: {start_date} - {end_date} ")
    main_interval=interval_dict[intervals[0]]
    curr_date=start_date
    main_interval.set_date(curr_date) # Setting the current date ( or nearest existing date) for interval
    if(main_interval.curr_date is None):
        logging.info(f"Empty data for interval {main_interval.interval} for date {curr_date}")
        return None, None
    
    curr_date=main_interval.curr_date # Uploading current date to existing in data
    while  curr_date <= end_date:
        #Set the current date for interval
        if PRINT_DAILY_UPDATE:
            logging.info(f"Starting iteration for date: {curr_date} ")
        #logging.info(f"Starting iteration for date: {curr_date} ")
        #main_interval.set_date(curr_date)
        row=main_interval.row
        if (row is None  or row['timestamp']!=curr_date): #ELack of data for current date. Skipping
            #logging.info(f" ERROR:  row timestamp: {row['timestamp']}  curr date: {curr_date} ")
            #print(f" ERROR:  row timestamp: {row['timestamp']}  curr date: {curr_date} ")
            curr_date += interval_mapping[main_interval.interval]['dataOffset']
            continue
            #return None, None
        if(main_interval.curr_date is None): #End of data
            logging.info(f"ERROR Empty data for interval {main_interval.interval} for date {curr_date}. End of data!")
            #print(f" ERROR:  row timestamp: {row['timestamp']}  curr date: {curr_date} ")

            return None, None
        #Processing orders
        if Active_orders:
            do_usuniecia = []
            for order in Active_orders:
                logging.info(f"Processing order: {order.order_type} {order.status} {order.create_time} ")
                if order.order_type=='OPEN MARKET' and order.status =='CREATED' and order.create_time<curr_date:
                    entry_time = curr_date
                    entry_price = main_interval.data.at[main_interval.curr_idx, 'open']
                    new_trade = order.fill(entry_time, entry_price)
                    do_usuniecia.append(order)
                    
                    if PRINT_LOGS:
                        logging.info(f"New trade created: {new_trade}")

                    Active_trades.append(new_trade)
                    balance -= new_trade.deposit

            for order in do_usuniecia:
                Active_orders.remove(order)
                if PRINT_LOGS:
                    logging.info(f"Order removed: {order} , BALANCE: {balance} AVAILABLE_BALANCE: {available_balance}")

        #Processing current trades. Looking for new trades ( setting orders)
        # LOOKING FOR TRADES
        if strategy=='0.0':  # Buy&Hold strategy
            if active_long==0: # Enetring long
                    deposit=calculate_deposit(initial_balance, available_balance)
                    if available_balance>=deposit:
                        new_order=create_market_open_order("L" , deposit, row.copy(), strategy)
                        Active_orders.append(new_order)
                        active_long=1
                        available_balance -= new_order.deposit 
        if strategy=='0.1':  # First strtategy: SMA, MACD, StochRSI
            if row['SMA_20'] > row['SMA_150']: # 
                if active_short==1: # Exit short
                    trade=Active_trades[0] # Only one trade
                    active_short=0
                    exit_price = row['close']  # Cena wyjścia
                    trade.set_exit( exit_time= curr_date, exit_price= exit_price)
                    available_balance += trade.exit_value
                    trade.result = "SIGNAL"
                    Active_trades.remove(trade)
                    Closed_trades.append(trade)
                    if PRINT_LOGS:
                        logging.info(f"Exited trade: {trade}, available balance: {available_balance}")
                        logging.info(f"row['D'] : {row['D'] }, row['MACD_Histogram'] {row['MACD_Histogram']}")

                if active_long==0 and row['SMA_20'] > row['SMA_150'] and row['D']  <= 20 and row['MACD_Histogram'] >0: # Enetring long
                    deposit=calculate_deposit(initial_balance, available_balance)
                    if available_balance>=deposit:
                        new_order=create_market_open_order("L" , deposit, row.copy(), strategy)
                        Active_orders.append(new_order)
                        active_long=1
                        available_balance -= new_order.deposit 
                        if PRINT_LOGS:
                            logging.info(f"New order created: {new_order} , available balance: {available_balance}")
                            
            if row['SMA_20'] < row['SMA_150']: # 
                if active_long==1: # Exit long
                    trade=Active_trades[0] # Only one trade
                    active_long=0
                    exit_price = row['close']  # Cena wyjścia
                    trade.set_exit( exit_time= curr_date, exit_price= exit_price)
                    trade.result = "SIGNAL"
                    available_balance += trade.exit_value
                    Active_trades.remove(trade)
                    Closed_trades.append(trade)
                    if PRINT_LOGS:
                        logging.info(f"Exited trade: {trade}, available balance: {available_balance}")

                if active_short==0 and row['SMA_20'] < row['SMA_150'] and row['D']  >=80  and row['MACD_Histogram'] <0: # Enetring short
                    deposit=calculate_deposit(initial_balance, available_balance)
                    if available_balance>=deposit:

                        new_order=create_market_open_order("S" , deposit, row.copy(), strategy)
                        Active_orders.append(new_order)
                        active_short=1
                        available_balance -= new_order.deposit 
                        if PRINT_LOGS:
                            logging.info(f"New order created: {new_order} , available balance: {available_balance}")

        if strategy=='0.2': # Second strategy: MACD
            if row['Signal_MACD']==1: # 
                if active_short==1: # Exit short
                    trade=Active_trades[0] # Only one trade
                    active_short=0
                    exit_price = row['close']  # Cena wyjścia
                    trade.set_exit( exit_time= curr_date, exit_price= exit_price)
                    available_balance += trade.exit_value
                    trade.result = "SIGNAL"
                    Active_trades.remove(trade)
                    Closed_trades.append(trade)
                    if PRINT_LOGS:
                        logging.info(f"Exited trade: {trade}, available balance: {available_balance}")
                        logging.info(f"row['D'] : {row['D'] }, row['MACD_Histogram'] {row['MACD_Histogram']}")

                if active_long==0 : # Enetring long
                    deposit=calculate_deposit(initial_balance, available_balance)
                    if available_balance>=deposit:
                        new_order=create_market_open_order("L" , deposit, row.copy(), strategy)
                        Active_orders.append(new_order)
                        active_long=1
                        available_balance -= new_order.deposit 
                        if PRINT_LOGS:
                            logging.info(f"New order created: {new_order} , available balance: {available_balance}")
                            
            if row['Signal_MACD']==-1: # 
                if active_long==1: # Exit long
                    trade=Active_trades[0] # Only one trade
                    active_long=0
                    exit_price = row['close']  # Cena wyjścia
                    trade.set_exit( exit_time= curr_date, exit_price= exit_price)
                    trade.result = "SIGNAL"
                    available_balance += trade.exit_value
                    Active_trades.remove(trade)
                    Closed_trades.append(trade)
                    if PRINT_LOGS:
                        logging.info(f"Exited trade: {trade}, available balance: {available_balance}")

                if active_short==0 : # Enetring short
                    deposit=calculate_deposit(initial_balance, available_balance)
                    if available_balance>=deposit:

                        new_order=create_market_open_order("S" , deposit, row.copy(), strategy)
                        Active_orders.append(new_order)
                        active_short=1
                        available_balance -= new_order.deposit 
                        if PRINT_LOGS:
                            logging.info(f"New order created: {new_order} , available balance: {available_balance}")



        if strategy=='0.3': # Third strategy: EMA(20), close. LONG oonly
            if row['close'] > row['EMA_20']: # 

                if active_long==0 : # Enetring long
                    deposit=calculate_deposit(initial_balance, available_balance)
                    if available_balance>=deposit:
                        new_order=create_market_open_order("L" , deposit, row.copy(), strategy)
                        Active_orders.append(new_order)
                        active_long=1
                        available_balance -= new_order.deposit 
                        if PRINT_LOGS:
                            logging.info(f"New order created: {new_order} , available balance: {available_balance}")
                            
            if row['close'] < row['EMA_20']: # 
                if active_long==1: # Exit long
                    trade=Active_trades[0] # Only one trade
                    active_long=0
                    exit_price = row['close']  # Cena wyjścia
                    trade.set_exit( exit_time= curr_date, exit_price= exit_price)
                    trade.result = "SIGNAL"
                    available_balance += trade.exit_value
                    Active_trades.remove(trade)
                    Closed_trades.append(trade)
                    if PRINT_LOGS:
                        logging.info(f"Exited trade: {trade}, available balance: {available_balance}")

        if strategy=='0.4': # Fourth strategy: EMA(10), SMA_100, RSI for entering Short
            if row['EMA_10']>=row['SMA_100']: # 
                if active_short==1: # Exit short
                    trade=Active_trades[0] # Only one trade
                    active_short=0
                    exit_price = row['close']  # Cena wyjścia
                    trade.set_exit( exit_time= curr_date, exit_price= exit_price)
                    available_balance += trade.exit_value
                    trade.result = "SIGNAL"
                    Active_trades.remove(trade)
                    Closed_trades.append(trade)
                    if PRINT_LOGS:
                        logging.info(f"Exited trade: {trade}, available balance: {available_balance}")
                        logging.info(f"row['D'] : {row['D'] }, row['MACD_Histogram'] {row['MACD_Histogram']}")

                if active_long==0 : # Enetring long
                    deposit=calculate_deposit(initial_balance, available_balance)
                    if available_balance>=deposit:
                        new_order=create_market_open_order("L" , deposit, row.copy(), strategy)
                        Active_orders.append(new_order)
                        active_long=1
                        available_balance -= new_order.deposit 
                        if PRINT_LOGS:
                            logging.info(f"New order created: {new_order} , available balance: {available_balance}")
                            
            if row['EMA_10']<row['SMA_100']: # 
                if active_long==1: # Exit long
                    trade=Active_trades[0] # Only one trade
                    active_long=0
                    exit_price = row['close']  # Cena wyjścia
                    trade.set_exit( exit_time= curr_date, exit_price= exit_price)
                    trade.result = "SIGNAL"
                    available_balance += trade.exit_value
                    Active_trades.remove(trade)
                    Closed_trades.append(trade)
                    if PRINT_LOGS:
                        logging.info(f"Exited trade: {trade}, available balance: {available_balance}")

                if active_short==0 and row['RSI']<50 : # Enetring short, extra condition RSI<50
                    deposit=calculate_deposit(initial_balance, available_balance)
                    if available_balance>=deposit:

                        new_order=create_market_open_order("S" , deposit, row.copy(), strategy)
                        Active_orders.append(new_order)
                        active_short=1
                        available_balance -= new_order.deposit 
                        if PRINT_LOGS:
                            logging.info(f"New order created: {new_order} , available balance: {available_balance}")

        if strategy=='0.5': # Fifth strategy: Williams%R
            if row['Williams_%R'] <-80: # 
                if active_short==1: # Exit short
                    trade=Active_trades[0] # Only one trade
                    active_short=0
                    exit_price = row['close']  # Cena wyjścia
                    trade.set_exit( exit_time= curr_date, exit_price= exit_price)
                    available_balance += trade.exit_value
                    trade.result = "SIGNAL"
                    Active_trades.remove(trade)
                    Closed_trades.append(trade)
                    if PRINT_LOGS:
                        logging.info(f"Exited trade: {trade}, available balance: {available_balance}")
                        logging.info(f"row['D'] : {row['D'] }, row['MACD_Histogram'] {row['MACD_Histogram']}")

                if active_long==0 : # Enetring long
                    deposit=calculate_deposit(initial_balance, available_balance)
                    if available_balance>=deposit:
                        new_order=create_market_open_order("L" , deposit, row.copy(), strategy)
                        Active_orders.append(new_order)
                        active_long=1
                        available_balance -= new_order.deposit 
                        if PRINT_LOGS:
                            logging.info(f"New order created: {new_order} , available balance: {available_balance}")
                            
            if row['Williams_%R'] >-20: # 
                if active_long==1: # Exit long
                    trade=Active_trades[0] # Only one trade
                    active_long=0
                    exit_price = row['close']  # Cena wyjścia
                    trade.set_exit( exit_time= curr_date, exit_price= exit_price)
                    trade.result = "SIGNAL"
                    available_balance += trade.exit_value
                    Active_trades.remove(trade)
                    Closed_trades.append(trade)
                    if PRINT_LOGS:
                        logging.info(f"Exited trade: {trade}, available balance: {available_balance}")

                if active_short==0 : # Enetring short
                    deposit=calculate_deposit(initial_balance, available_balance)
                    if available_balance>=deposit:

                        new_order=create_market_open_order("S" , deposit, row.copy(), strategy)
                        Active_orders.append(new_order)
                        active_short=1
                        available_balance -= new_order.deposit 
                        if PRINT_LOGS:
                            logging.info(f"New order created: {new_order} , available balance: {available_balance}")


        if strategy=='0.6': # Sixth strategy: Williams%R, Candle pattern
            if   (row['Williams_%R_lag'] < -80 or row['Williams_%R'] < -80) : #
                if active_short==1: # Exit short
                    trade=Active_trades[0] # Only one trade
                    active_short=0
                    exit_price = row['close']  # Cena wyjścia
                    trade.set_exit( exit_time= curr_date, exit_price= exit_price)
                    available_balance += trade.exit_value
                    trade.result = "SIGNAL"
                    Active_trades.remove(trade)
                    Closed_trades.append(trade)
                    if PRINT_LOGS:
                        logging.info(f"Exited trade: {trade}, available balance: {available_balance}")
                        logging.info(f"row['D'] : {row['D'] }, row['MACD_Histogram'] {row['MACD_Histogram']}")

                if active_long==0 and row['Bullish_Pattern'] == 1  : # Enetring long
                    deposit=calculate_deposit(initial_balance, available_balance)
                    if available_balance>=deposit:
                        new_order=create_market_open_order("L" , deposit, row.copy(), strategy)
                        Active_orders.append(new_order)
                        active_long=1
                        available_balance -= new_order.deposit 
                        if PRINT_LOGS:
                            logging.info(f"New order created: {new_order} , available balance: {available_balance}")
                            
            if  (row['Williams_%R_lag'] >-20 or row['Williams_%R'] >-20) : # 
                if active_long==1 and Active_trades: # Exit long
                    trade=Active_trades[0] # Only one trade
                    active_long=0
                    exit_price = row['close']  # Cena wyjścia
                    trade.set_exit( exit_time= curr_date, exit_price= exit_price)
                    trade.result = "SIGNAL"
                    available_balance += trade.exit_value
                    Active_trades.remove(trade)
                    Closed_trades.append(trade)
                    if PRINT_LOGS:
                        logging.info(f"Exited trade: {trade}, available balance: {available_balance}")

                if active_short==0 and row['Bearish_Pattern'] == 1  : # Enetring short
                    deposit=calculate_deposit(initial_balance, available_balance)
                    if available_balance>=deposit:

                        new_order=create_market_open_order("S" , deposit, row.copy(), strategy)
                        Active_orders.append(new_order)
                        active_short=1
                        available_balance -= new_order.deposit 
                        if PRINT_LOGS:
                            logging.info(f"New order created: {new_order} , available balance: {available_balance}")


        # Calculate account worth. Save to account history
        account_worth=available_balance

        for trade in Active_trades:
            account_worth+=trade.calculate_current_value(row['close'])
        for order in Active_orders:
            account_worth+=order.deposit
        account_history = update_account_history(account_history, curr_date, account_worth)
        if account_worth<start_balance*0.1:
            break
                    # Set the next date
        curr_date += interval_mapping[main_interval.interval]['dataOffset']
        main_interval.set_next_date()


     # END OF BACKTESTING
     # CLOSING ACTIVE TRADES
             
    if Active_trades:
        for trade in Active_trades[:]:  
            trade.set_exit(exit_time=curr_date, exit_price=row['close'])
            trade.result = "END"
            available_balance += trade.exit_value

            if PRINT_LOGS:
                logging.info(f"Exited trade: {trade}, available balance: {available_balance}")

            Active_trades.remove(trade)
            Closed_trades.append(trade)

    closed_trades_df= pd.DataFrame()

    if Closed_trades:
        closed_trades_df = pd.DataFrame([trade.to_dict() for trade in Closed_trades])
    
        closed_trades_df = closed_trades_df.sort_values(by='Entry Time', ascending=True)



    return closed_trades_df, account_history


def setup_simulation_logger(output_path, logs_folder, ticker, strategy_id, interval):
    # Ensure log folder exists
    log_dir = os.path.join(output_path, logs_folder)
    os.makedirs(log_dir, exist_ok=True)

    # Create log file path
    log_file = os.path.join(log_dir, f"{ticker} - strategy {strategy_id}, interval {interval} - logs.txt")

    # Remove all handlers associated with the root logger object (to avoid duplicate logs)
    for handler in logging.root.handlers[:]:
        logging.root.removeHandler(handler)

    # Configure logging
    logging.basicConfig(
        filename=log_file,
        level=logging.INFO,
        format='%(asctime)s - %(levelname)s - %(message)s'
    )

    return log_file  # Optional: return path to log file


In [ ]:
# Load market data and strategy configuration files
market_data_table = pd.read_excel(DATA_TABLE_FILE)
strategy_table = pd.read_excel(STRATEGIES_TABLE_FILE)

# Define input/output directories
data_input_folder = MARKET_DATA_DICT
output_folder = SIMULATION_DATA_DICT



def run_simulations(intervals, start_date, end_date):
    start_dt = pd.to_datetime(start_date)
    end_dt = pd.to_datetime(end_date)
    output_path = os.path.join(output_folder, f"{start_date}_to_{end_date}")
    os.makedirs(output_path, exist_ok=True)
    for interval in intervals:
        for strat_idx, strat_row in strategy_table.iterrows():
            strategy_id = str(strat_row['Numer'])

            for data_idx, data_row in market_data_table.iterrows():
                ticker = data_row['Ticker']
                # if ticker != "BTC":  # Limit to BTC for now
                #     continue

                ticker_file = data_row['File']

                # Define output file paths
                trades_output_file = f"{output_path}/{ticker} - strategy {strategy_id}, interval {interval} - trades.xlsx"
                history_output_file = f"{output_path}/{ticker} - strategy {strategy_id}, interval {interval} - history.xlsx"

                # Skip if results already exist
                if os.path.exists(trades_output_file) and os.path.exists(history_output_file):
                    if strategy_id != '0.0':
                        continue

                print(f"Processing: Interval = {interval}, Strategy = {strategy_id} ticker: {data_idx}")

                # Load and prepare local market data
                raw_data = get_local_data(ticker_file, interval, data_input_folder)
                raw_data['timestamp'] = pd.to_datetime(raw_data['timestamp'], format="%Y-%m-%d")

                # Adjust date range if out of bounds
                start_sim = max(start_dt, raw_data['timestamp'].iloc[0])
                end_sim = min(end_dt, raw_data['timestamp'].iloc[-1])

                # Create interval object and initialize

                setup_simulation_logger(output_path, LOGS_FOLDER, ticker, strategy_id, interval)
                # Process data and run backtest
                processed_data = process_data(raw_data)
                interval_instance = Interval(ticker, interval, processed_data)
                interval_dict = {interval: interval_instance}
                logging.info(f"Logging started for simulation: {ticker}, strategy {strategy_id}, interval {interval}")
                trades, history = backtest(
                    interval_dict=interval_dict,
                    ticker=ticker,
                    strategy=strategy_id,
                    intervals=[interval],
                    start_date=start_sim,
                    end_date=end_sim,
                    start_balance=START_BALANCE,
                    position_risk=RISK_SIZE,
                    stop_loss=False
                )

                # Save results to Excel
                history.to_excel(history_output_file, index=False)
                trades.to_excel(trades_output_file, index=False)








# Create output path including date range

# Define simulation date range
# Define intervals to simulate
intervals = ['1w', '3d', '2d', '1d', '12h', '6h', '4h', '2h', '1h', '30m', '15m']
start_date = "2010-01-01"
end_date = "2025-03-01"


start_date = "2021-11-01"
end_date = "2025-03-01"
period_id = '3'
# Convert dates to pandas datetime

run_simulations(intervals, start_date, end_date)
# Loop through intervals and strategies


Processing: Interval = 1w, Strategy = 0.0 ticker: 0
Processing: Interval = 1w, Strategy = 0.0 ticker: 1
Processing: Interval = 1w, Strategy = 0.0 ticker: 2
Processing: Interval = 1w, Strategy = 0.0 ticker: 3
Processing: Interval = 1w, Strategy = 0.0 ticker: 4
Processing: Interval = 1w, Strategy = 0.0 ticker: 5
Processing: Interval = 1w, Strategy = 0.0 ticker: 6
Processing: Interval = 1w, Strategy = 0.0 ticker: 7
Processing: Interval = 1w, Strategy = 0.0 ticker: 8
Processing: Interval = 1w, Strategy = 0.0 ticker: 9
Processing: Interval = 1w, Strategy = 0.0 ticker: 10
Processing: Interval = 1w, Strategy = 0.0 ticker: 11
Processing: Interval = 1w, Strategy = 0.0 ticker: 12
Processing: Interval = 1w, Strategy = 0.0 ticker: 13
Processing: Interval = 1w, Strategy = 0.0 ticker: 14
Processing: Interval = 1w, Strategy = 0.0 ticker: 15
Processing: Interval = 1w, Strategy = 0.0 ticker: 16
Processing: Interval = 1w, Strategy = 0.0 ticker: 17
Processing: Interval = 1w, Strategy = 0.0 ticker: 18
Pro